In [ ]:
#建立句子 平均長度為9 測試版
import pymysql

connection = pymysql.connect(host="140.119.164.170", user="salan40319", passwd="104753018", db="FreeChina",charset='utf8')
file = []
for i in range(1):
    if i < 9:
        file.append('0'+str(i+1)+'_All')
    else:
        file.append(str(i+1)+'_All')
print (file)

g = 0


try:
    for j in file:
        with connection.cursor() as cursor:
            sql = "select * from `{}`"

            cursor.execute(sql.format(j))

            temp_line = ''
            #( 「 『 』

            c = 0
            check = False

            l = 0
            e = 0

            for i in cursor:
                temp_line = temp_line + i[1].decode('utf8')
                if len(temp_line) == 1 and i[2].decode('utf8') == '(PARENTHESISCATEGORY)':
                    e = e + 1
                    if i[1].decode('utf8') == '」' or i[1].decode('utf8') == '』':
                        check = True
                if i[2].decode('utf8') == '(PARENTHESISCATEGORY)':
                    c = c + 1

                if c == 2 and len(temp_line) == 2:
                    temp_line = ''
                    c = 0
                    check = False

                if i[2].decode('utf8') == '(PERIODCATEGORY)' or i[2].decode('utf8') == '(COMMACATEGORY)':
                    if len(temp_line) > 8:
                        if not check:
                            print ('*'+temp_line) 
                            pass
                        else:
                            print ('#'+temp_line[1:]) 
                            pass
                        temp_line = ''
                        l = l + 1
                        check = False
                        c = 0

            #print (l,e) #7529 168
            g = g + l

finally:
    connection.close()

print (g)

In [ ]:
#建立句子 14775203字
import pymysql
import codecs

connection = pymysql.connect(host="140.119.164.170", user="salan40319", passwd="104753018", db="FreeChina",charset='utf8')

#得到所有table
file = []
for i in range(23):
    if i < 9:
        file.append('0'+str(i+1)+'_All')
    else:
        file.append(str(i+1)+'_All')
print (file)

check = False #判斷是否到達文章結尾
line_index = 0

try:
    for i in file:
        with codecs.open("C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\FreeChina\\"+i+'.txt','w','utf8') as f:
            with connection.cursor() as cursor:
                sql = "select * from `{}`"

                cursor.execute(sql.format(i))

                temp_line = ''
                #( 「 『 』
                
                print (i)

                for i in cursor:

                    if check:
                        check = False
                        #若為以下字元開頭，補進上一欄
                        if (i[1].decode('utf8') == '』' or i[1].decode('utf8') == '」' 
                            or i[1].decode('utf8') == '）' or i[1].decode('utf8') == ')'):
                            temp_line = temp_line + i[1].decode('utf8') + i[2].decode('utf8')
                            #print ('*'+temp_line)
                            f.write (temp_line.strip()+'\r\n')
                            temp_line = ''
                            continue
                        #將上一行印出
                        else:
                            #print ('*'+temp_line)
                            f.write (temp_line.strip()+'\r\n')
                            temp_line = ''
                        line_index =line_index + 1
                    #判斷是否為datetime 獨立一行
                    if i[2].decode('utf8') == '(DATETIME)':
                        if len(temp_line) != 0:
                            #print ('#'+temp_line)
                            f.write (temp_line.strip()+'\r\n')
                            line_index = line_index + 1
                        temp_line = '# '


                    temp_line = temp_line + i[1].decode('utf8') + i[2].decode('utf8') + ' '
                    
                    #判斷是否為。，？！；：結尾，author則為獨立一行
                    if (i[2].decode('utf8') == '(PERIODCATEGORY)' or i[2].decode('utf8') == '(COMMACATEGORY)' 
                        or i[2].decode('utf8') == '(QUESTIONCATEGORY)' or i[2].decode('utf8') == '(EXCLAMATIONCATEGORY)'
                        or i[2].decode('utf8') == '(SEMICOLONCATEGORY)' or i[2].decode('utf8') == '(COLONCATEGORY)'
                        or i[2].decode('utf8') == '(AUTHOR)'):
                        check = True
                
                if len(temp_line) != 0:
                    #print ('*'+temp_line)
                    f.write (temp_line.strip()+'\r\n')
                    line_index = line_index + 1
                    
                check = False

finally:
    connection.close()
    
print (line_index)
print ('END')

In [ ]:
#提取分析 1273711行 3533區分段 14775203字數
import codecs

path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\FreeChina\\{}_All.txt"

files = []
for i in range(23):
    if i < 9:
        files.append('0'+str(i+1))
    else:
        files.append(str(i+1))

t = 0      
line_index = 0
        
for file in files:
    with codecs.open(path.format(file),'r','utf8') as f:
        
        temp = f.readlines()
        
        for content in temp:
            line_index = line_index + 1
        
            if content[0] == '#':
                
                content = content.split()
                
                content[2] = ''.join(content[2:5])
                del content[3:5]
                del content[0]
                if len(content) > 4:
                    content[2] = ''.join(content[2:-1])
                    del content[3:-1]
                    #print (content)
                else:
                    #print (content)
                    pass
                
                t = t + 1
print (t)
print (line_index)
   

In [ ]:
#情態動詞分析 1273711行 
import codecs
import time

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
path = "C:\\Users\\user\\Desktop\\FreeChina\\{}_All.txt"

files = []
for i in range(23):
    if i < 9:
        files.append('0'+str(i+1))
    else:
        files.append(str(i+1))

two_verb = {}
        
for file in files:
    with codecs.open(path.format(file),'r','utf8') as f:
        
        content = f.readlines()
        
        for line in content:
            
            if line[0] != '#':
                
                words = line.split()
                verb_list = []
                
                for word in words:
                    if word[0] == '(' or word[0] == '（':
                        word = [word[0],word[1:]]
                    else:
                        word = word.split('(')
                        word[1] = '('+word[1]

                    if word[0] in modalverb:
                        verb_list.append(''.join(word))

                if len(verb_list) >= 2:
                    for l in range(len(verb_list)-1):
                        k = l + 1
                        if (verb_list[l],verb_list[k]) in two_verb:
                            two_verb[(verb_list[l],verb_list[k])] = two_verb[(verb_list[l],verb_list[k])] + 1
                        else:
                            two_verb[(verb_list[l],verb_list[k])] = 1

two_verb_count = 0

for i in two_verb:
    two_verb_count = two_verb_count + two_verb[i]
    
two_verb = sorted(two_verb.items(), key=lambda d:d[1], reverse = True)

print (len(two_verb)) #488
print (two_verb_count) #6418

for i in two_verb:
    print (i[0],i[1])


In [10]:
#情態動詞尋找 1273711行 
#02 15457 16080兩段資料似乎重複
#有引用句也會導致重複
import codecs
import time
from colorama import init

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\FreeChina\\{}_All.txt"

files = []
for i in range(23):
    if i < 9:
        files.append('0'+str(i+1))
    else:
        files.append(str(i+1))
        
find = ['該(D)', '會(D)']

two_verb = {}
        
for file in files:
    with codecs.open(path.format(file),'r','utf8') as f:
        
        content = f.readlines()
        print (file)
        
        for line in content:
            
            if line[0] != '#':
                
                words = line.split()
                verb_list = []
                verb_index = []
                verb_index_a = []
                check = False
                index = 0
                
                for word in words:
                    if word[0] == '(' or word[0] == '（':
                        word = [word[0],word[1:]]
                    else:
                        word = word.split('(')
                        word[1] = '('+word[1]

                    if word[0] in modalverb:
                        verb_list.append(''.join(word))
                        verb_index.append(index)
                    index = index + 1

                if len(verb_list) >= 2:
                    for l in range(len(verb_list)-1):
                        k = l + 1
                        if find[0] == verb_list[l] and find[1] == verb_list[k]:
                            check = True
                            if verb_index[l] not in verb_index_a:
                                verb_index_a.append(verb_index[l])
                            if verb_index[k] not in verb_index_a:
                                verb_index_a.append(verb_index[k])
                
                for i in verb_index_a:
                    words[i] = '\033[31;46m' + words[i] + '\033[0m'
                            
                
                if check:
                    print (''.join(words))
                    time.sleep(0.3)
                    
                    


01
為(P)使(VL)該(D)會(D)能(D)成(VG)維持(VJ)和平(VH)有效(VH)的(DE)工具(Na)﹐(COMMACATEGORY)
02
我們(Nh)就(D)該(D)會(D)知道(VK)，(COMMACATEGORY)
該(D)會(D)領導(VC)者(Na)蔣(Nb)總統(Na)夫人(Na)發表(VC)了(Di)「(PARENTHESISCATEGORY)今日(Nd)中華(Nc)婦女(Na)的(DE)主要(A)使命(Na)」(PARENTHESISCATEGORY)一(Neu)文(Nf)。(PERIODCATEGORY)
03
但(Cbb)今日(Nd)該(D)會(D)已經(D)不能(D)發生(VJ)絲毫(Neqa)的(DE)效用(Na)。(PERIODCATEGORY)
該(D)會(D)擁有(VJ)會員(Na)四十五萬(Neu)人(Na)﹐(COMMACATEGORY)
該(D)會(D)的(DE)任務(Na)完成(VC)﹐(COMMACATEGORY)
嚴煙(Nb)更(D)不(D)是(SHI)首傳(Na)該(D)會(D)入(VCL)台(Nc)之(DE)人(Na)。(PERIODCATEGORY)
04
該(D)會(D)共有(VJ)委員(Na)七十一(Neu)人(Na)，(COMMACATEGORY)
國防(Na)都(D)長(VC)馬歇爾(Nb)又(D)出席(VC)該(D)調查(VE)會(D)對(P)麥帥(Nb)證詞(Na)作答(VA)；(SEMICOLONCATEGORY)
05
我們(Nh)大家(Nh)想想(VE)其(Nep)後果(Na)該(D)會(D)怎樣(VH)？(QUESTIONCATEGORY)
該(D)會(D)制(VC)有(V_2)標準(Na)漢字表(Na)，(COMMACATEGORY)
其後(Ncd)該(D)會(D)復(D)於(P)昭和(Nd)十七年(Nd)，(COMMACATEGORY)
據說(Dk)該(D)會(D)並(D)計劃(VF)向(P)印度(Nc)再行(D)選精(Na)若干(Neqa)，(COMMACATEGORY)
他(Nh)該(D)會(D)多麼(Dfa)，(COMMACATEGORY)


KeyboardInterrupt: 

In [1]:
import codecs
import os
from ipywidgets import IntProgress
from IPython.display import display

p = IntProgress()
p.max = 23
p.description = 'start'
display(p)

modalverb = ["應","要","可","能","可以","須","應該","必須","會","得","需要","當","應當","能夠","該","需"]
path = "C:\\Users\\user\\Desktop\\SNA\\meeting\\source\\FreeChina\\"
out_path = "C:\\Users\\user\\Desktop\\"

file_list = []
count = 0

for file in os.listdir(path):
    file_list.append(file)
    
dic = {}
sen = {}

for file in file_list:
    with codecs.open(path+file,'rb','utf8') as f:
        contents = f.readlines()

        for i in contents:
            if i[0] == '#':
                continue

            temp = i.strip().split()

            index = 0
            pairs = []

            for j in temp:
                words = j.split('(')
                words[1] = '(' + words[1]

                if words[0] in modalverb:
                    if index != len(temp)-1:
                        '''if temp[index+1].split('(')[1][:-1] != 'COMMACATEGORY' and  \
                            temp[index+1].split('(')[1][:-1] != 'PERIODCATEGORY' and \
                            temp[index+1].split('(')[1][:-1] != 'QUESTIONCATEGORY':'''
                        if (words[1] == '(D)' or words[1][1] == 'V') and \
                            (temp[index+1].split('(')[1][0] == 'V' or temp[index+1].split('(')[1][0] == 'D' \
                            or temp[index+1].split('(')[1][0] == 'P' or temp[index+1].split('(')[1][0] == 'C'):
                            pairs.append(temp[index])
                index += 1

            if len(pairs) > 1:
                
                for j in range(len(pairs)-1):    
                    term = pairs[j]+'_'+pairs[j+1]
                
                    if term not in dic:
                        dic[term] = 1
                    else:
                        dic[term] += 1

                    if term not in sen:
                        sen[term] = [''.join(temp)]
                    elif len(sen[term]) < 3:
                        sen[term].append(''.join(temp))
                        
    count = count + 1
    p.value = count
    p.description = str(count)

dic = sorted(dic.items(), key=lambda d:d[1], reverse = True)
with codecs.open(out_path+'output.txt','wb','utf8') as f:
    for i in dic:
        #print (i[0],i[1])
        f.write(str(i[0])+' '+str(i[1]))
        '''for j in sen[i[0]]:
            f.write(j+'\r\n')'''
        f.write('\r\n')

    
p.description = 'end'